In [1]:
%pip install pinecone-client openai tiktoken

Note: you may need to restart the kernel to use updated packages.


In [1]:
from langchain.document_loaders import PyPDFLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.schema import Document
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Pinecone
import os
import getpass
import pinecone

/Users/jamesaphoenix/opt/anaconda3/lib/python3.9/site-packages/pinecone/index.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [ ]:
os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter in your OpenAI API Key:")

In [ ]:
PINECONE_API_KEY = getpass.getpass("Pinecone API Key:")
PINECONE_ENV = getpass.getpass("Pinecone Environment:")

In [2]:
# loader = PyPDFLoader("data/principles_of_marketing_book.pdf") # If the file was local
loader = PyPDFLoader(
    "https://storage.googleapis.com/strapi_cms_assets/principles_of_marketing_book.pdf"
)
raw_documents = loader.load_and_split()
print(raw_documents[0])

page_content='Principles of Mark eting' metadata={'source': '/var/folders/_y/20jl658s4jl0zvy5c0x0c5140000gn/T/tmpr74w14jh/tmp.pdf', 'page': 0}


In [3]:
# Using the last 100 documents for this example:
raw_documents = raw_documents[-100:]

In [7]:
# Load the marketing principles .pdf, split it into chunks, embed each chunk and load it into the vector store.
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
documents = text_splitter.split_documents(raw_documents)

# Initialize the Pinecone authentication:
pinecone.init(
    api_key="24d00f96-93a9-4e6f-a3db-c9039146087d", environment="us-west4-gcp-free"
)

# Re-write the pinecone.init to use the variables we set above:
# pinecone.init(api_key=PINECONE_API_KEY, environment=PINECONE_ENV)

index_name = "langchain-test"
index = pinecone.Index(index_name=index_name)

In [11]:
# This will create a new vector store, only run this line once or you will duplicate the vector store and the number of documents in it:
doc_search = Pinecone.from_documents(
    documents, embedding=OpenAIEmbeddings(), index_name=index_name
)

Often, you won't want to ingest data into your vector database, but simply query the existing vectors:


In [18]:
existing_vector_store = Pinecone.from_existing_index(
    index_name=index_name, embedding=OpenAIEmbeddings()
)
retriever = existing_vector_store.as_retriever()